In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(7578752, 55)

In [3]:
# add observation index
data["row_index"] = data.index

In [4]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (7519013, 56)
test: (39821, 56)
known: (7558834, 56)
unknown: (19918, 56)


In [5]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff_release_listen',
       'release_year', 'is_listened_lag1', 'is_listened_lag2',
       'user_skip_ratio_last5', 'genre_equal_last_song',
       'artist_equal_last_song', 'album_equal_last_song', 'genre_plays',
       'genre_skips', 'artist_plays', 'artist_skips', 'album_plays',
       'album_skips', 'song_plays', 'song_skips', 'user_ratio_flow',
       'user_ratio_full', 'genre_ratio', 'artist_ratio', 'song_ratio',
       'user_genre_ratio', 'user_artist_ratio', 'user_song_ratio',
       'platform_name', 'platform_family', 'hour_of_day1', 'hour_of_day2',
       'hour_of_day3', 'hour_of_day4', 'hour_of_day5', 'hour_of_day6',
       'hour_of_day7', 'hour_of_day8', 'weekdayMon', 'weekdaySat',
       'weekdaySun', 'weekdayThu', 'weekdayTue

## 2. MODELING

### 2.1. INITIALIZING

In [6]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   50, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 50, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([u, s, a, c])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
x = Dropout(0.5)(embedding_dense)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([user_in, song_in, artist_in, context_in], output)
model.compile(optimizer = "adamax", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [7]:
# run the estimations on training data
model.fit([tr.user_id, tr.media_id, tr.artist_id, tr.context_type], tr.is_listened, 
validation_data = ([ts.user_id, ts.media_id, ts.artist_id, ts.context_type], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 7519013 samples, validate on 39821 samples
Epoch 1/10
7519013/7519013 [==============================] - 166s - loss: 0.5966 - acc: 0.7222 - val_loss: 0.6870 - val_acc: 0.6076
Epoch 2/10
7519013/7519013 [==============================] - 163s - loss: 0.5278 - acc: 0.7597 - val_loss: 0.6679 - val_acc: 0.6110
Epoch 3/10
7519013/7519013 [==============================] - 165s - loss: 0.5126 - acc: 0.7650 - val_loss: 0.6519 - val_acc: 0.6344
Epoch 4/10
7519013/7519013 [==============================] - 154s - loss: 0.5017 - acc: 0.7689 - val_loss: 0.6359 - val_acc: 0.6636
Epoch 5/10
7519013/7519013 [==============================] - 153s - loss: 0.4929 - acc: 0.7725 - val_loss: 0.6225 - val_acc: 0.6772
Epoch 6/10
7519013/7519013 [==============================] - 156s - loss: 0.4853 - acc: 0.7760 - val_loss: 0.6191 - val_acc: 0.6792
Epoch 7/10
7519013/7519013 [==============================] - 153s - loss: 0.4786 - acc: 0.7792 - val_loss: 0.6247 - val_acc: 0.6783
Epoch 8/10
751901

In [8]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.user_id, ts.media_id, ts.artist_id, ts.context_type])
pred.to_csv(path + "pred_valid/keras_newdata_full_song_artist_context_50_nonum_adamax_10ep.csv", index = False)
pred.head(5)

,row_index,is_listened
5989,5989,0.983996
6053,6053,0.981035
6296,6296,0.995374
11840,11840,0.989487
11910,11910,0.999499


In [9]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.72209150152645885

### 2.3. SECOND STAGE

In [10]:
# run the estimations on full known data
model.fit([kn.user_id, kn.media_id, kn.artist_id, kn.context_type], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
7558834/7558834 [==============================] - 158s - loss: 0.4609 - acc: 0.7883   
Epoch 2/10
7558834/7558834 [==============================] - 155s - loss: 0.4574 - acc: 0.7902   
Epoch 3/10
7558834/7558834 [==============================] - 155s - loss: 0.4541 - acc: 0.7920   
Epoch 4/10
7558834/7558834 [==============================] - 154s - loss: 0.4510 - acc: 0.7935   
Epoch 5/10
7558834/7558834 [==============================] - 152s - loss: 0.4484 - acc: 0.7949   
Epoch 6/10
7558834/7558834 [==============================] - 152s - loss: 0.4458 - acc: 0.7964   
Epoch 7/10
7558834/7558834 [==============================] - 152s - loss: 0.4433 - acc: 0.7976   
Epoch 8/10
7558834/7558834 [==============================] - 152s - loss: 0.4413 - acc: 0.7988   
Epoch 9/10
7558834/7558834 [==============================] - 153s - loss: 0.4391 - acc: 0.7998   
Epoch 10/10
7558834/7558834 [==============================] - 153s - loss: 0.4371 - acc: 0.8009   


In [11]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.user_id, un.media_id, un.artist_id, un.context_type])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_full_song_artist_context_50_nonum_adamax_10ep.csv", index = False)
pred.head(5)

,sample_id,is_listened
7551765,0,0.999753
6913499,1,0.817787
6529339,2,0.851174
5409309,3,0.666113
6218646,4,0.815577
